In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sql_api").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 18:59:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [39]:
from pyspark.sql.functions import col, regexp_replace, split, explode, length, expr

In [3]:
df = spark.read.csv("movies.dat", sep='::', schema='MovieID int, oldTitle string, combinedGenres string')

In [56]:
# Split title_year column into title, year
df = df.withColumn("len", length(col("oldtitle")))
final_df = df.withColumn("Title", expr("substring(oldTitle, 1, len - 7)")).withColumn("Year", expr("substring(oldTitle, len - 4, 4)")).withColumn("Genres", split("combinedGenres", '\\|'))
final_df = final_df.drop("oldTitle", "oldTitle_Modified", "Title_Year", "combinedGenres", "len")
movies_df = final_df.select("MovieID", "Title", "Year", explode("Genres").alias("Genre"))
movies_df.write.saveAsTable("movies", mode="overwrite")
# movies_df.show()

In [57]:
movies_df = spark.read.table("movies")

## Movies by genre

In [58]:
def get_movies_by_genre(genre, num):
    movies = movies_df.select("Title").filter(movies_df["Genre"] == genre)
    return movies.rdd.flatMap(lambda x: x).take(num)

In [59]:
print(get_movies_by_genre("Adventure", 20))

['Jumanji', 'Tom and Huck', 'GoldenEye', 'Cutthroat Island', 'City of Lost Children, The', 'Wings of Courage', 'Mortal Kombat', 'Kids of the Round Table', 'Indian in the Cupboard, The', 'White Squall', 'Muppet Treasure Island', 'Rumble in the Bronx', 'NeverEnding Story III, The', 'Amazing Panda Adventure, The', 'Batman Forever', 'Casper', 'Congo', 'First Knight', 'Free Willy 2: The Adventure Home', 'Judge Dredd']


In [60]:
def get_movies_count_by_genre(genre):
    movies = movies_df.select("Title").filter(movies_df["Genre"] == genre)
    return movies.rdd.flatMap(lambda x: x).count()

In [61]:
print(get_movies_count_by_genre("Adventure"))

283


In [62]:
def get_movies_by_year(year, num):
    movies = movies_df.select("Title").filter(movies_df["Year"] == int(year)).distinct()
    return movies.rdd.flatMap(lambda x: x).take(num)

In [63]:
print(get_movies_by_year("1989", 20))

['Fright Night Part II', 'Honey, I Shrunk the Kids', 'Lethal Weapon 2', 'Police Academy 6: City Under Siege', 'Batman', 'For All Mankind', 'Do the Right Thing', 'My Left Foot', 'Indiana Jones and the Last Crusade', 'Christmas Vacation', 'Henry V', 'Killer, The (Die xue shuang xiong)', "We're No Angels", 'Crimes and Misdemeanors', 'Prancer', 'Drugstore Cowboy', 'Star Trek V: The Final Frontier', 'Abyss, The', 'Gods Must Be Crazy II, The', 'Communion']


In [64]:
def get_movies_count_by_year(year):
    movies = movies_df.select("Title").filter(movies_df["Year"] == int(year)).distinct()
    return movies.rdd.flatMap(lambda x: x).count()

In [65]:
get_movies_count_by_year(1989)

60

In [1]:
print("hello")

hello
